In [ ]:
import site
import os
import json
from datetime import datetime
from qiskit.circuit.parameterexpression import ParameterExpression
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.fake_provider import fake_backend

class BackendEncoder(json.JSONEncoder):
    """A json encoder for qobj"""

    def default(self, o):
        # Convert numpy arrays:
        if hasattr(o, "tolist"):
            return o.tolist()
        # Use Qobj complex json format:
        if isinstance(o, complex):
            return [o.real, o.imag]
        if isinstance(o, ParameterExpression):
            return float(o)
        if isinstance(o, datetime):
            return o.isoformat()
        return json.JSONEncoder.default(self, o)

class FakeBackend(fake_backend.FakeBackendV2):
    
    def __init__(self, backend):

        package_name = 'qiskit_ibm_runtime'
        package_dirs = site.getsitepackages()
        backends_dir = None
        for dir in package_dirs:
            package_path = os.path.join(dir, package_name)
            if os.path.exists(package_path):
                backends_dir = os.path.join(package_path, "fake_provider", "backends")
                break

        backend_og_name = backend.name.split("_")[1]
        self.conf_filename = f"conf_{backend_og_name}.json" 
        self.props_filename = f"props_{backend_og_name}.json" 
        self.defs_filename = f"defs_{backend_og_name}.json"
        self.backend_name = f"fake_{backend_og_name}"
        self.dirname = os.path.join(backends_dir, backend_og_name)
        if not os.path.exists(self.dirname):
            os.mkdir(self.dirname)
            config = backend.configuration()
            props = backend.properties()
            defs = backend.defaults()
            if config:
                config_path = os.path.join(self.dirname, self.conf_filename)
                config_dict = config.to_dict()
                with open(config_path, "w") as fd:
                    fd.write(json.dumps(config_dict, cls=BackendEncoder))
            if props:
                props_path = os.path.join(self.dirname, self.props_filename)
                with open(props_path, "w") as fd:
                    fd.write(json.dumps(props.to_dict(), cls=BackendEncoder))
            if defs:
                defs_path = os.path.join(self.dirname, self.defs_filename)
                with open(defs_path, "w") as fd:
                    fd.write(json.dumps(defs.to_dict(), cls=BackendEncoder))

        super().__init__()

In [ ]:
service = QiskitRuntimeService()
instance = service.instances()[0] # Or put your instance manually
print(instance)
backend = service.get_backend("ibm_torino")

In [ ]:
fake_backend = FakeBackend(backend)